In [1]:
import minsearch
import json
from openai import OpenAI

In [2]:
with open("./data/documents.json", "rt") as f_in:
    docs_raw = json.load(f_in)

In [3]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict["documents"]:
        doc["course"] = course_dict["course"]
        documents.append(doc)

In [4]:
index = minsearch.Index(
    text_fields = ["question", "text", "section"],
    keyword_fields = ["course"]
  ) 

In [5]:
q = "The course has already started, can I still enroll?"

In [6]:
index.fit(documents)

In [7]:
client = OpenAI(
    base_url = "http://localhost:11434/v1",
    api_key = "ollama"
)

In [8]:
def search(query):
    boost = {"question": 3.0, "section": 0.5}

    results = index.search(
        query=query,
        filter_dict={"course": "data-engineering-zoomcamp"},
        boost_dict=boost,
        num_results=5
    )
    return results

In [9]:
def build_prompt(query, search_results):
    prompt_template = """
You are a teaching assistant for a course. Answer the QUESTION, based on the CONTEXT.
Use only the facts from the CONTEXT, when answering the QUESTION.
Keep the answer succinct and to the point.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
    context = ""

    for doc in search_results:
        context += f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [10]:
def llm(prompt):
    response = client.chat.completions.create(
    model='mistral',
    messages=[{'role': 'user', 'content': prompt}]
)
    return response.choices[0].message.content

In [11]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [13]:
print(rag("The course has already started, can I still enroll?"))

 Based on the provided context, yes, you can still enroll for the course after it has started, however, there will be deadlines for turning in the final projects. The course materials will also be kept for future reference so that you can follow the course at your own pace even after it finishes. It is advisable to install and set up all dependencies and requirements before the course starts, such as Google cloud account, Google Cloud SDK, Python 3 (installed with Anaconda), Terraform, Git. The Slack channel remains open for support if you take the course in self-paced mode. It is suggested to search for answers in the Slack channel and the FAQ document before asking questions on the platform or to the bot @ZoomcampQABot. The exact date of the course start is 15th Jan 2024 at 17h00.


In [14]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [15]:
from elasticsearch import Elasticsearch

In [17]:
es_client = Elasticsearch('http://localhost:9200')

In [19]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"}
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [20]:
from tqdm.auto import tqdm

In [21]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [22]:
query = "I just discovered the course, can I still join it?"

In [29]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    
    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [31]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [ ]:
print(rag(query))